In [ ]:
%pip install --upgrade pip
%pip install tensorflow
%pip install tensorflow-gpu
%pip install matplotlib
%pip install scipy
%pip install pandas
%pip install opencv_python
%pip install numpy
%pip install scikit-learn

In [4]:
import os

train_path = "../../datasets/personal/clean/75x75/train/"
test_path  = "../../datasets/personal/clean/75x75/test/"

classes = os.listdir(train_path)
classes = sorted(classes)
n_classes = len(classes)
print(f"Classes: {classes}")

input_shape = (75, 75, 3)  # InceptionV3 input size
file_count = []

for fld in os.listdir(train_path):
    crt = os.path.join(train_path, fld)
    image_count = len(os.listdir(crt))
    file_count.append(image_count)
    print(f'{crt} contains {image_count} images')

print(f'\nTotal number of images: {sum(file_count)}')

Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y']
../../datasets/personal/clean/75x75/train/A contains 300 images
../../datasets/personal/clean/75x75/train/B contains 300 images
../../datasets/personal/clean/75x75/train/C contains 300 images
../../datasets/personal/clean/75x75/train/D contains 300 images
../../datasets/personal/clean/75x75/train/E contains 300 images
../../datasets/personal/clean/75x75/train/F contains 300 images
../../datasets/personal/clean/75x75/train/G contains 300 images
../../datasets/personal/clean/75x75/train/I contains 300 images
../../datasets/personal/clean/75x75/train/L contains 300 images
../../datasets/personal/clean/75x75/train/M contains 300 images
../../datasets/personal/clean/75x75/train/N contains 300 images
../../datasets/personal/clean/75x75/train/O contains 300 images
../../datasets/personal/clean/75x75/train/P contains 300 images
../../datasets/personal/clean/75x75/train/Q contains 

In [5]:
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input

# Define channel place
keras.backend.set_image_data_format('channels_last')

## Define constants
BATCH_SIZE = 32
SEED = 123

## Data augmentation and preprocessing
# Training Gen
train_data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_data_generator.flow_from_directory(
    train_path,
    target_size=input_shape[:2],
    shuffle=True,
    seed=SEED,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    subset="training"
)

# Validation Gen
val_data_generator = ImageDataGenerator(
    validation_split=0.2,
    preprocessing_function=preprocess_input,
)

validation_generator = val_data_generator.flow_from_directory(
    train_path,
    target_size=input_shape[:2],
    shuffle=False,
    seed=SEED,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    subset="validation"
)

# Test Gen
test_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)
test_generator = test_generator.flow_from_directory(
    test_path,
    target_size=input_shape[:2],
    shuffle=False,
    seed=SEED,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
)

## Image quantity
train_samples = train_generator.samples
validation_samples = validation_generator.samples
test_samples = test_generator.samples

Found 5040 images belonging to 21 classes.
Found 1260 images belonging to 21 classes.
Found 4200 images belonging to 21 classes.


In [6]:
import matplotlib.pyplot as plt

# Visualize Examples
plt.figure(figsize=(8,8))

for i in range(9):
    plt.subplot(330 + 1 + i)
    batch = train_generator.next()[0]*255
    image = batch[0].astype('uint8')
    plt.imshow(image)

plt.show()

In [8]:
from keras.applications import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

# Load the InceptionV3 model with pre-trained weights (excluding top layers)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

# Freezing layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for your specific task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(n_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)
model.load_weights('../InceptionV3/libras/inception_v3_weights.h5')
model.summary() 

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 75, 75, 3)]          0         []                            
                                                                                                  
 conv2d_94 (Conv2D)          (None, 37, 37, 32)           864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, 37, 37, 32)           96        ['conv2d_94[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_94 (Activation)  (None, 37, 37, 32)           0         ['batch_normalization_94

In [9]:
EPOCHS = 2
filepath='../InceptionV3/personal/'
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath=f'{filepath}inception_v3_weights.h5',
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=f'{filepath}inception_v3_model.h5',
        monitor='val_loss',
        mode='min',
        save_best_only=True,
        save_weights_only=False,
        verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=30,
        restore_best_weights=True,
        verbose=1,
    ),
]

# Compiling Model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

#Training
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=train_samples//BATCH_SIZE,
    callbacks = callbacks_list,
    validation_data=validation_generator,
    verbose = 1,
    validation_steps=validation_samples//BATCH_SIZE,
)

Epoch 1/2
157/157 [==============================] - ETA: 0s - loss: 0.7798 - accuracy: 0.7486
Epoch 1: val_accuracy improved from -inf to 0.91827, saving model to ../InceptionV3/personal\inception_v3_weights.h5

Epoch 1: val_loss improved from inf to 0.40686, saving model to ../InceptionV3/personal\inception_v3_model.h5


c:\Users\Ricardo\Documents\GitHub\Pessoal\TCC\python-tcc\training\.venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


157/157 [==============================] - 39s 207ms/step - loss: 0.7798 - accuracy: 0.7486 - val_loss: 0.4069 - val_accuracy: 0.9183
Epoch 2/2
157/157 [==============================] - ETA: 0s - loss: 0.4014 - accuracy: 0.8674
Epoch 2: val_accuracy improved from 0.91827 to 0.92869, saving model to ../InceptionV3/personal\inception_v3_weights.h5

Epoch 2: val_loss improved from 0.40686 to 0.37647, saving model to ../InceptionV3/personal\inception_v3_model.h5
157/157 [==============================] - 28s 178ms/step - loss: 0.4014 - accuracy: 0.8674 - val_loss: 0.3765 - val_accuracy: 0.9287
